# Predicción de Rotación de Personal (HR Turnover Prediction)

Este notebook analiza datos de empleados para predecir el riesgo de rotación utilizando un modelo de Random Forest. El análisis incluye:
- Carga de datos desde SQL Server
- Preprocesamiento de datos
- Entrenamiento del modelo
- Evaluación de resultados
- Almacenamiento de predicciones

## 1. Importación de Bibliotecas

Importamos las bibliotecas necesarias para el análisis.

In [ ]:
import pandas as pd
import pymssql
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
import datetime
import logging
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

# Configuración de logging
logging.basicConfig(level=logging.INFO)

## 2. Conexión a Base de Datos y Consulta

Configuramos la conexión a SQL Server y ejecutamos la consulta para obtener los datos de empleados.

In [ ]:
# Configuración de SQL Server
SQL_SERVER = "172.28.192.1:50121\SQLEXPRESS"
SQL_DB = "HR_Analytics"
SQL_USER = "sa"
SQL_PASSWORD = "123456"

# Conectar a SQL Server usando pymssql
server_name = SQL_SERVER.split('\\')[0]  # Separar el nombre del servidor de la instancia
conn = pymssql.connect(
    server=server_name,
    user=SQL_USER,
    password=SQL_PASSWORD,
    database=SQL_DB
)

query = """
SELECT w.employee_id, w.department, w.salary, w.hire_date, 
       AVG(k.hours_worked) as avg_hours_worked, 
       AVG(k.overtime_hours) as avg_overtime_hours, 
       AVG(s.satisfaction_score) as avg_satisfaction_score,
       CASE WHEN w.status = 'Terminated' THEN 1 ELSE 0 END AS turnover
FROM Workday_Employees w
LEFT JOIN Kronos_TimeEntries k ON w.employee_id = k.employee_id
LEFT JOIN Employee_Surveys s ON w.employee_id = s.employee_id
GROUP BY w.employee_id, w.department, w.salary, w.hire_date, w.status
"""

df = pd.read_sql(query, conn)
print(f"Datos cargados: {len(df)} registros")
df.head(10)

In [ ]:
### QUALITY REPORT (PRE CLEANING)

def create_data_quality_report(df):
    # Initialize dictionary to store quality metrics
    quality_report = {
        'Column_Name': [],
        'Data_Type': [],
        'Total_Rows': [],
        'Missing_Values': [],
        'Missing_Percentage': [],
        'Blank_Values': [],
        'Blank_Percentage': [],
        'Zero_Values': [],
        'Zero_Percentage': [],
        'Unique_Values': []
    }
    
    total_rows = len(df)
    
    for column in df.columns:
        # Count missing values
        missing_count = df[column].isna().sum()
        
        # Count blank values (empty strings)
        blank_count = df[column].astype(str).str.strip().eq('').sum()
        
        # Count zero values for numeric columns
        if pd.api.types.is_numeric_dtype(df[column]):
            zero_count = (df[column] == 0).sum()
        else:
            zero_count = 0
            
        # Count unique values
        unique_count = df[column].nunique()
        
        # Add to report
        quality_report['Column_Name'].append(column)
        quality_report['Data_Type'].append(str(df[column].dtype))
        quality_report['Total_Rows'].append(total_rows)
        quality_report['Missing_Values'].append(missing_count)
        quality_report['Missing_Percentage'].append(round(missing_count/total_rows * 100, 2))
        quality_report['Blank_Values'].append(blank_count)
        quality_report['Blank_Percentage'].append(round(blank_count/total_rows * 100, 2))
        quality_report['Zero_Values'].append(zero_count)
        quality_report['Zero_Percentage'].append(round(zero_count/total_rows * 100, 2))
        quality_report['Unique_Values'].append(unique_count)
    
    # Create DataFrame from the quality report
    quality_df = pd.DataFrame(quality_report)
    
    # Sort by missing percentage (descending)
    quality_df = quality_df.sort_values('Missing_Percentage', ascending=False)
    
    return quality_df

try:
    # Verify if df exists and is a DataFrame
    if not isinstance(df, pd.DataFrame):
        raise NameError("df is not a valid DataFrame")
    
    # Create the quality report
    quality_report = create_data_quality_report(df)

    selected_columns = df.columns.tolist()
    Q_Report = quality_report[quality_report['Column_Name'].isin(selected_columns)]
except NameError:
    print("Error: Please make sure to run the previous cell that creates the 'df' DataFrame first.")
    print("The DataFrame should be created in the cell that executes the SQL query.")

Q_Report

## 3. Preprocesamiento de Datos

Preparamos los datos para el entrenamiento del modelo:
- Calculamos la antigüedad (tenure)
- Codificamos variables categóricas
- Tratamos valores faltantes

In [ ]:
# Calcular antigüedad
df['tenure'] = (pd.to_datetime('today') - pd.to_datetime(df['hire_date'])).dt.days / 365

# Manejo específico de avg_overtime_hours
df['avg_overtime_hours'] = df['avg_overtime_hours'].fillna(0)

# Manejo condicional de avg_hours_worked
df.loc[(df['avg_hours_worked'].isna()) & (df['tenure'] > 1), 'avg_hours_worked'] = 8
df.loc[(df['avg_hours_worked'].isna()) & (df['tenure'] <= 1), 'avg_hours_worked'] = 0

# Imputación de avg_satisfaction_score usando XGBoost
def impute_satisfaction_scores(df):
    # Separar datos para entrenamiento (registros con satisfaction_score)
    train_mask = ~df['avg_satisfaction_score'].isna()
    features = ['tenure', 'avg_hours_worked', 'avg_overtime_hours', 'salary']
    
    X_train = df[train_mask][features]
    y_train = df[train_mask]['avg_satisfaction_score']
    
    # Preparar datos para predicción
    X_predict = df[~train_mask][features]
    
    # Crear y entrenar modelo XGBoost para imputación
    imputer = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42
    )
    
    # Entrenar modelo
    imputer.fit(X_train, y_train)
    
    # Evaluar precisión del modelo
    train_predictions = imputer.predict(X_train)
    mse = mean_squared_error(y_train, train_predictions)
    r2 = r2_score(y_train, train_predictions)
    
    print("\nMétricas del modelo de imputación:")
    print(f"Error cuadrático medio (MSE): {mse:.4f}")
    print(f"R² Score: {r2:.4f}")
    
    # Realizar predicciones para valores faltantes
    predictions = imputer.predict(X_predict)
    
    # Asignar predicciones a los valores faltantes
    df.loc[~train_mask, 'avg_satisfaction_score'] = predictions
    
    return df

# Aplicar imputación
df = impute_satisfaction_scores(df)

# Codificar variables categóricas
df = pd.get_dummies(df, columns=['department'], drop_first=True)

print("\nEstadísticas descriptivas después del preprocesamiento:")
print(df.describe())

# Verificar valores faltantes restantes
missing_values = df.isnull().sum()
print("\nValores faltantes restantes:")
print(missing_values[missing_values > 0])

In [ ]:
### QUALITY REPORT (POST CLEANING)

def create_data_quality_report(df):
    # Initialize dictionary to store quality metrics
    quality_report = {
        'Column_Name': [],
        'Data_Type': [],
        'Total_Rows': [],
        'Missing_Values': [],
        'Missing_Percentage': [],
        'Blank_Values': [],
        'Blank_Percentage': [],
        'Zero_Values': [],
        'Zero_Percentage': [],
        'Unique_Values': []
    }
    
    total_rows = len(df)
    
    for column in df.columns:
        # Count missing values
        missing_count = df[column].isna().sum()
        
        # Count blank values (empty strings)
        blank_count = df[column].astype(str).str.strip().eq('').sum()
        
        # Count zero values for numeric columns
        if pd.api.types.is_numeric_dtype(df[column]):
            zero_count = (df[column] == 0).sum()
        else:
            zero_count = 0
            
        # Count unique values
        unique_count = df[column].nunique()
        
        # Add to report
        quality_report['Column_Name'].append(column)
        quality_report['Data_Type'].append(str(df[column].dtype))
        quality_report['Total_Rows'].append(total_rows)
        quality_report['Missing_Values'].append(missing_count)
        quality_report['Missing_Percentage'].append(round(missing_count/total_rows * 100, 2))
        quality_report['Blank_Values'].append(blank_count)
        quality_report['Blank_Percentage'].append(round(blank_count/total_rows * 100, 2))
        quality_report['Zero_Values'].append(zero_count)
        quality_report['Zero_Percentage'].append(round(zero_count/total_rows * 100, 2))
        quality_report['Unique_Values'].append(unique_count)
    
    # Create DataFrame from the quality report
    quality_df = pd.DataFrame(quality_report)
    
    # Sort by missing percentage (descending)
    quality_df = quality_df.sort_values('Missing_Percentage', ascending=False)
    
    return quality_df

try:
    # Verify if df exists and is a DataFrame
    if not isinstance(df, pd.DataFrame):
        raise NameError("df is not a valid DataFrame")
    
    # Create the quality report
    quality_report = create_data_quality_report(df)

    selected_columns = df.columns.tolist()
    Q_Report = quality_report[quality_report['Column_Name'].isin(selected_columns)]
except NameError:
    print("Error: Please make sure to run the previous cell that creates the 'df' DataFrame first.")
    print("The DataFrame should be created in the cell that executes the SQL query.")

Q_Report

## 4.1 Selección de Features con RFE y XGBoost


In [ ]:
## 4.1 Selección de Features con RFE y XGBoost

# Preparar datos para RFE
# Identificar columnas numéricas y dummy de departamentos
numeric_features = ['salary', 'tenure', 'avg_hours_worked', 'avg_overtime_hours', 'avg_satisfaction_score']
department_features = [col for col in df.columns if col.startswith('department_')]
all_features = numeric_features + department_features

print("Features totales para RFE:")
print(all_features)

X_rfe = df[all_features]
y_rfe = df['turnover']

# Escalar solo las variables numéricas
scaler = StandardScaler()
X_rfe[numeric_features] = scaler.fit_transform(X_rfe[numeric_features])

# Crear modelo XGBoost para RFE
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

# Aplicar RFE con más features a seleccionar para incluir departamentos
rfe_selector = RFE(
    estimator=xgb_model,
    n_features_to_select=8,  # Aumentamos el número de features a seleccionar
    step=1,
    verbose=1
)

rfe_selector.fit(X_rfe, y_rfe)

# Mostrar resultados de RFE
feature_ranking = pd.DataFrame({
    'Feature': all_features,
    'Selected': rfe_selector.support_,
    'Ranking': rfe_selector.ranking_
}).sort_values('Ranking')

print("\nResultados de la selección RFE:")
print(feature_ranking)

# Separar features seleccionados por tipo
selected_features = feature_ranking[feature_ranking['Selected']]['Feature'].tolist()
selected_numeric = [f for f in selected_features if f in numeric_features]
selected_departments = [f for f in selected_features if f in department_features]

print("\nFeatures numéricos seleccionados:")
print(selected_numeric)
print("\nDepartamentos seleccionados:")
print(selected_departments)

## 5. Entrenamiento del Modelo

Entrenamos el modelo usando los features seleccionados por RFE y datos escalados.

In [ ]:
# Usar features seleccionados por RFE
X = df[selected_features]
y = df['turnover']

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

# Definir parámetros para Grid Search con XGBoost
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 3],
    'gamma': [0, 0.1]
}

# Crear modelo base XGBoost
xgb_base = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    random_state=42
)

# Realizar Grid Search con validación cruzada
grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

# Entrenar modelo con Grid Search
print("Iniciando búsqueda de mejores hiperparámetros...")
grid_search.fit(X_train_balanced, y_train_balanced)

# Imprimir mejores parámetros
print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)

# Usar directamente el mejor modelo del GridSearch
model = grid_search.best_estimator_

# Entrenar el modelo final
model.fit(X_train_balanced, y_train_balanced)

# Realizar predicciones
y_pred = model.predict(X_test_scaled)

# Calcular y mostrar métricas detalladas
print("\nReporte de clasificación detallado:")
print(classification_report(y_test, y_pred))

# Mostrar matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusión:")
print(conf_matrix)

# Visualizar matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.ylabel('Verdaderos')
plt.xlabel('Predichos')
plt.show()

# Mostrar importancia de características
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

# Visualizar importancia de características
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Importancia de Características')
plt.xlabel('Importancia')
plt.tight_layout()
plt.show()

# Calcular probabilidades
y_pred_proba = model.predict_proba(X_test_scaled)

# Encontrar el mejor umbral de decisión
thresholds = np.arange(0.1, 0.9, 0.1)
best_threshold = 0.5
best_f1 = 0

for threshold in thresholds:
    y_pred_threshold = (y_pred_proba[:, 1] >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred_threshold)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"\nMejor umbral de decisión: {best_threshold:.2f}")
print(f"F1-Score con mejor umbral: {best_f1:.4f}")

# Actualizar predicciones con el mejor umbral
final_predictions = (y_pred_proba[:, 1] >= best_threshold).astype(int)

# Calcular métricas finales
metrics = {
    'accuracy': accuracy_score(y_test, final_predictions),
    'precision': precision_score(y_test, final_predictions, zero_division=0),
    'recall': recall_score(y_test, final_predictions, zero_division=0),
    'f1_score': f1_score(y_test, final_predictions, zero_division=0)
}

print("\nMétricas finales con umbral optimizado:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

## 6. Guardar Modelo y Métricas

Guardamos el modelo entrenado y almacenamos las métricas en la base de datos.

In [ ]:
# Crear directorio si no existe
os.makedirs('Modelos Entrenados', exist_ok=True)

# Guardar modelo
model_path = 'Modelos Entrenados/turnover_model.pkl'
joblib.dump(model, model_path)
print(f"Modelo guardado en: {model_path}")

# Guardar métricas en SQL Server
cursor = conn.cursor()
cursor.execute(
    """
    INSERT INTO ML_Model_Accuracy (model_name, run_date, accuracy, precision, recall, f1_score)
    VALUES (%s, %s, %s, %s, %s, %s)
    """,
    ('Turnover_Prediction', datetime.datetime.now(), metrics['accuracy'],
     metrics['precision'], metrics['recall'], metrics['f1_score'])
)
conn.commit() # Added commit to save change

## 7. Guardar Predicciones

Generamos y guardamos las probabilidades de rotación para todos los empleados.

In [ ]:
# Calcular probabilidades y predicciones de rotación
X_scaled = scaler.transform(X)  # Escalar datos antes de predecir
df['turnover_probability'] = model.predict_proba(X_scaled)[:, 1]
df['predicted_turnover'] = (df['turnover_probability'] >= best_threshold).astype(int)

# Guardar predicciones en la base de datos
cursor = conn.cursor()
for _, row in df.iterrows():
    # Primero eliminar registros existentes
    cursor.execute(
        "DELETE FROM Turnover_Predictions WHERE employee_id = %s",
        (row['employee_id'],)
    )
    
    # Luego insertar el nuevo registro
    cursor.execute(
        """
        INSERT INTO Turnover_Predictions 
        (employee_id, prediction_date, turnover_probability, predicted_turnover)
        VALUES (%s, %s, %s, %s)
        """,
        (row['employee_id'], datetime.datetime.now(), row['turnover_probability'], row['predicted_turnover'])
    )

conn.commit() # Added commit to save change


## 8. Cerrar conexión

Cerrando conexión con el backend.

In [ ]:
conn.commit()
print("Predicciones guardadas exitosamente")

# Mostrar las 10 predicciones más altas
print("\nTop 10 empleados con mayor riesgo de rotación:")
high_risk = df[['employee_id', 'turnover_probability', 'predicted_turnover']].sort_values('turnover_probability', ascending=False).head(10)
print(high_risk)

# Mostrar distribución de predicciones
print("\nDistribución de predicciones de rotación:")
print(df['predicted_turnover'].value_counts(normalize=True).round(3) * 100, "%")

# Cerrar conexión
conn.close()
print("\nConexión a la base de datos cerrada")

In [ ]:
df.tail(25)